In [164]:
import pandas as pd
from functools import reduce
import numpy as np
import math as math
from os import listdir

In [14]:
df1 = pd.read_csv("./csv/events_up_to_01062018.csv")

/home/tito/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
df2 = pd.read_csv("./csv/labels_training_set.csv")

In [16]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

In [17]:
def modelo(obj):
    if(pd.isnull(obj)):
        return obj
    
    list = obj.split(" ")
    if len(list) == 1:
        return obj
    if "iPhone" in list:
        return "iPhone"
    if "Samsung" in list:
        return "Samsung"
    if "Motorola" in list:
        return "Motorola"
    if "Lenovo" in list:
        return "Lenovo"
    if "Sony" in list:
        return "Sony"
    if "LG" in list:
        return "LG"
    if "Asus" in list:
        return "Asus"
    if "iPad" in list:
        return "iPad"
    if "Quantum" in list:
        return "Quantum"
    return obj

df1['model'] = df1['model'].apply(modelo)

In [18]:
df1['model'].value_counts()

iPhone              593603
Samsung             501307
Motorola            162914
LG                   24814
Sony                 15599
Lenovo               11382
iPad                  5663
Asus                  4536
Quantum               1693
Outros TV LED 15         1
Xiaomi Redmi 2           1
Name: model, dtype: int64

In [19]:
df1['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

# Analisis para los eventos distintos de convercion

In [20]:
df_a = df1

In [21]:
#calculo medianas de los modelos con eventos por persona
df_a_model = df_a.dropna(subset=['model']).loc[:,['person','model']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model_no_convercion"})


df_a_storage = df_a.dropna(subset=['storage']).loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage_no_convercion"})
        
        
df_a_condition = df_a.dropna(subset=['condition']).loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition_no_convercion"})
        
        
df_a_color = df_a.dropna(subset=['color']).loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color_no_convercion"})

df_a_search_engine = df_a.dropna(subset=['search_engine']).loc[:,['person','search_engine']].groupby('person')\
                .apply(lambda x: x['search_engine'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"search_engine"})

In [22]:
df_datos_no_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\
                                .merge(df_a_search_engine,how='left',on='person')

In [23]:
dffinal = df_datos_no_convercion

# Date

# eventos por dia x usr ( al temporario)

In [24]:
# creo columnas con lunes == true o false
def dds(ele):# 0 -> lunes
    if(ele.month == 1):
        return (((ele.day + 6) % 7) == (i))
    if(ele.month == 2):
        return (((ele.day + 3 + 6) % 7) == i)
    if(ele.month == 3):
        return (((ele.day + 3 + 6) % 7) ==(i))
    if(ele.month == 4):
        return (((ele.day + 12) % 7) ==(i))
    if(ele.month == 5):
        return (((ele.day + 1 + 6) % 7) == (i))
i = 0
df1["lunes"] = df1["timestamp"].apply(dds)
i = 1
df1["martes"] = df1["timestamp"].apply(dds)
i = 2
df1["miercoles"] = df1["timestamp"].apply(dds)
i = 3
df1["jueves"] = df1["timestamp"].apply(dds)
i = 4
df1["viernes"] = df1["timestamp"].apply(dds)
i = 5
df1["sabado"] = df1["timestamp"].apply(dds)
i = 6
df1["domingo"] = df1["timestamp"].apply(dds)


In [25]:
dffinal = dffinal.set_index("person")

# mayor cantidad de eventos separados por dia de la semaan por usr 

In [26]:
# mas actividad x dia semana
def diaMasGrande(fila):
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        aux = fila["domingo"]
    return aux
dffinal["numero mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(diaMasGrande,axis = 1)

# dia de la semana con mas eventos por usr

In [27]:
# dia de la semaan mas activa
def strDiaMasGrande(fila):
    strAux = "lunes"
    aux = fila["lunes"]
    if(aux < fila["martes"]):
        strAux = "martes"
        aux = fila["martes"]
    if(aux < fila["miercoles"]):
        strAux = "miercoles"
        aux = fila["miercoles"]
    if(aux < fila["jueves"]):
        strAux = "jueves"
        aux = fila["jueves"]
    if(aux < fila["viernes"]):
        strAux = "viernes"
        aux = fila["viernes"]
    if(aux < fila["sabado"]):
        strAux = "sabado"
        aux = fila["sabado"]
    if(aux < fila["domingo"]):
        strAux = "domingo"
        aux = fila["domingo"]
    return strAux
dffinal["nombre mas usado dia"] = df1[["lunes","martes","miercoles","jueves","viernes","sabado","domingo","person"]].groupby("person").sum().apply(strDiaMasGrande,axis = 1)

# cuento eventos por persona

In [28]:
aux = df1[["event","person"]]
aux["aux"] = 1
aux = aux.groupby(["person","event"]).count()
aux = aux.unstack().fillna(0).reset_index().reset_index()
aux.columns = aux.columns.droplevel()
aux["person"] = aux.iloc[:, 1].values
aux = aux.drop("",axis = 1)

/home/tito/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
dffinal = dffinal.reset_index()

In [30]:
aux = pd.merge(aux,dffinal,on="person",how="inner")

In [31]:
aux.columns

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'person',
       'model_no_convercion', 'storage_no_convercion',
       'condition_no_convercion', 'color_no_convercion', 'search_engine',
       'numero mas usado dia', 'nombre mas usado dia'],
      dtype='object')

In [32]:
aux = aux.dropna(subset=["storage_no_convercion"])

In [33]:
aux["search_engine"] = aux["search_engine"].fillna("not")

In [34]:
aux["storage_no_convercion"] = aux["storage_no_convercion"].str.replace("GB","").replace("512MB","0.5")

# ultimo evento/fecha de actividad/de de la semana activo
ultimos evento y su fecha y dias de la semana

In [35]:
temp = df1[["person","timestamp"]].groupby("person").max().reset_index()
aux["ultima_fecha_de_actividad"] = temp["timestamp"]

In [36]:
diasDeLaSemana =["lunes","martes","miercoles","jueves","viernes","sabado","domingo"]
def dds2(ele):# 0 -> lunes
    if(ele.month == 1):
        for i in range(0,7):
            if(((ele.day + 6) % 7) == (i)):
                return diasDeLaSemana[i]  
    if(ele.month == 2):
        for i in range(0,7):
            if(((ele.day + 9) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 3):
        for i in range(0,7):
            if(((ele.day + 9) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 4):
        for i in range(0,7):
            if(((ele.day + 12) % 7) == (i)):
                return diasDeLaSemana[i]
    if(ele.month == 5):
        for i in range(0,7):
            if(((ele.day) % 7) == (i)):
                return diasDeLaSemana[i]
            
aux["ultimo_dia_de_la_semana_activo"] = aux["ultima_fecha_de_actividad"].apply(dds2)

In [37]:
temp = df1[["person","timestamp"]].groupby("person")["timestamp"].max().reset_index()

In [38]:
df2 = pd.merge( df1[["person","timestamp","event"]],temp,on=["person","timestamp"],how="right")
#personas con varios eventos en el mismo timestamp/ como elegir cual evento fue el ultimo? 
#-->uso moda x q no se me ocurre nada mejor
tmpmod = df2.groupby("person").agg(pd.Series.mode)

In [39]:
aux["ultimo_evento_registrado_cmode"] = tmpmod.reset_index()["event"]

In [40]:
aux.columns

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'person',
       'model_no_convercion', 'storage_no_convercion',
       'condition_no_convercion', 'color_no_convercion', 'search_engine',
       'numero mas usado dia', 'nombre mas usado dia',
       'ultima_fecha_de_actividad', 'ultimo_dia_de_la_semana_activo',
       'ultimo_evento_registrado_cmode'],
      dtype='object')

In [41]:
temp = pd.to_datetime("06.01.2018") - aux["ultima_fecha_de_actividad"]

In [42]:
#lo paso a dias hasta el 1 / 06
aux["dias_hasta"] = temp.reset_index()["ultima_fecha_de_actividad"].dt.days

In [49]:
word = df1["event"].unique()
word

array(['viewed product', 'searched products', 'ad campaign hit',
       'staticpage', 'checkout', 'search engine hit', 'conversion',
       'generic listing', 'brand listing', 'visited site', 'lead'],
      dtype=object)

In [51]:
# fecha ultimo evento
ultimosEveFecha = df1.loc[df1["event"] == "conversion"][["person","timestamp"]].groupby("person")["timestamp"].max().reset_index().set_index("person")
del ultimosEveFecha["timestamp"]

In [52]:
eventList = df1["event"].unique()
for word in eventList:
    ultimosEveFecha[word + "_last"] = df1.loc[df1["event"] == word][["person","timestamp"]].groupby("person").max()

In [53]:
for word in eventList:
    ultimosEveFecha[word + "_last"] = (pd.to_datetime("06.01.2018") - pd.to_datetime(ultimosEveFecha[word + "_last"])).dt.days

In [54]:
#testear diferentes fills
ultimosEveFecha = ultimosEveFecha.fillna(-1).reset_index()

In [55]:
aux = pd.merge(aux,ultimosEveFecha,on="person",how="left").fillna(-1)

In [92]:
#cantidad de dias en los q entro
df1["dias_hasta_1/6"] = (pd.to_datetime("06.01.2018") - df1["timestamp"]).dt.days

In [57]:
aux = aux.set_index("person")
aux["cantidad_de_dias_utilizado"] = df1[["person","dias_hasta_1/6"]].groupby("person")["dias_hasta_1/6"].nunique().reset_index().set_index("person")
aux = aux.reset_index()

In [64]:
eventList

array(['viewed product', 'searched products', 'ad campaign hit',
       'staticpage', 'checkout', 'search engine hit', 'conversion',
       'generic listing', 'brand listing', 'visited site', 'lead'],
      dtype=object)

In [93]:
def concatenador(item):
       return reduce(lambda x, y: x + y, item)

In [94]:
def enlistar(x):
    return [x]
df1["dias_hasta_1/6"] = df1["dias_hasta_1/6"].apply(enlistar)

In [287]:
aux["fechas_de_conversiones"] = df1[["event","person","dias_hasta_1/6"]].loc[df1["event"] == "conversion"].groupby("person")["dias_hasta_1/6"].agg(concatenador).reset_index()["dias_hasta_1/6"]

In [288]:
aux["fechas_de_busqueda"] = df1[["event","person","dias_hasta_1/6"]].loc[df1["event"] == "searched products"].groupby("person")["dias_hasta_1/6"].agg(concatenador).reset_index()[0]

In [168]:
def isNaN(num):
    return num != num

In [290]:

def haceMagia(item):
    y = item.values
    x = y[0]
    y = y[1]
    l = []
    i = 10000
    if(not(isNaN(x)) and not(isNaN(y))):
        if(x and y ):
            x.append(i)
            x.sort(reverse = True)
            xanterior = -1
            l2 = []
            for numberx in x:
                if(xanterior != -1):
                    for numbery in y:
                        if(xanterior > numbery and numberx < numbery):
                            l2.append(numbery)
                xanterior = numberx  
            l.append(l2)
    return l

In [307]:
def len2(x):
    if(len(x) == 2):
        print(1)
aux[["fechas_de_conversiones","fechas_de_busqueda"]].fillna(0).apply(haceMagia,axis = 1)

0                                                     [[]]
1                                                     [[]]
2        [[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, ...
3                                                     [[]]
4            [[31, 31, 31, 31, 31, 31, 31, 111, 111, 111]]
5                                               [[14, 14]]
6                                                       []
7                                                     [[]]
8                                                     [[]]
9                                               [[32, 32]]
10                                                    [[]]
11       [[107, 107, 107, 107, 107, 107, 107, 107, 107,...
12                  [[135, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10]]
13                                                    [[]]
14                                                    [[]]
15                                                    [[]]
16                          [[21, 21, 21, 21, 21, 74, 74

In [65]:
df1[["event","person","timestamp"]].loc[df1["event"] == "searched products"].groupby()

(130616, 31)

In [58]:
del aux["ultima_fecha_de_actividad"]
aux.to_csv("features.csv",index = False)